In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using CairoMakie, GLMakie
using LinearAlgebra, LinearSolve

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

In [3]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305

t = 1000; tt = 1000; tstep = 0.001; tspan = [time, tt]
integ_set = (alg = Vern9(), adaptive = true, abstol = 1e-15, reltol = 1e-15);

In [4]:
index_hom = 1
I0 = I0_hom[index_hom]; U0 = u0_hom[index_hom]
p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [11.325905642223786, 0.6594706953104683, 0.4863175026548461];

In [5]:
ds = CoupledODEs(TM, u0, p, diffeq = integ_set)
fp, _, _ = fixedpoints(ds, box, jacob_TM_)
ϵ_sep = 1e-7
ϵ_box = 1e-10
Ju0 = jacob_TM_(fp[1], p, 0);

In [35]:
function condition(u, t, integrator)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf)
end
affect!(integrator) = terminate!(integrator)

affect! (generic function with 1 method)

In [6]:
eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors

v1 = real(eigen_vectors[:, 1])
v2 = real(eigen_vectors[:, 2])
v3 = imag(eigen_vectors[:, 3])

v1 = normalize(v1, Inf)
v2 = normalize(v2, Inf)
v3 = normalize(v3, Inf)

v1t = transpose(v1)
v2t = transpose(v2)
v3t = transpose(v3)

dot_on_sep_stable = fp[1] + v1*ϵ_sep
A = transpose([v1t; v2t; v3t])
A = Matrix(A)

3×3 Matrix{Float64}:
  1.0         1.0          0.0
 -0.0490503  -0.0141828    1.0
  0.0119273   0.000705923  0.176983

In [7]:
ds = CoupledODEs(TM, dot_on_sep_stable, p, diffeq = integ_set)

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  TM
 ODE solver:    Vern9
 ODE kwargs:    (adaptive = true, abstol = 1.0e-15, reltol = 1.0e-15)
 parameters:    [1.58, 0.013, 0.07993, 3.3, 3.07, 0.75, 0.4, 0.2650000065028337, 0.305, 0.3, -1.70629651132633]
 time:          0.0
 state:         [8.345808551620786, 0.7384946597477061, 0.43829856154578317]


In [8]:
tr, trange = trajectory(ds, t, Ttr = tt, Δt = tstep)
println("Length sol: $(length(tr))");

Length sol: 1000001


In [9]:
αs = zeros(3, length(tr))
norminfs = zeros(length(tr));

In [10]:
for (i, dot) in enumerate(tr)

    b = dot - fp[1]
    b = Vector(b)
    prob = LinearProblem(A, b)
    sol = solve(prob)
    
    """vec = [abs(sol.u[1]), abs(sol.u[2]), abs(sol.u[3])]
    norminf = maximum(vec)"""

    αs[:, i] = sol
    norminfs[i] = norm(sol.u, Inf)
end

In [11]:
minimum(norminfs)- ϵ_box

4.899478947192118e-5

In [12]:
minimum(αs)

-13.059739281578095

In [13]:
"""ts, tf = 100000, 200000
idx, idy, idz = 2, 3, 1
dis  = 60;

f = Figure(resolution = (700, 700))
ax = LScene(f[1, 1], show_axis = true)
scale!(ax.scene, 50, 50, 1)

lines!(tr[ts:tf, idx], tr[ts:tf, idy], tr[ts:tf, idz], linewidth = 1.0, color = :black)
for (i, ep) in enumerate(fp)
    scatter!(ep[idx], ep[idy], ep[idz], color = :blue, markersize = 8)
end

display(GLMakie.Screen(), f)""";